In [ ]:
library(arrow)
library(aws.s3)
library(haven) # read_sav

# Configuração do S3
Sys.setenv(
    "AWS_ACCESS_KEY_ID" = "",
    "AWS_SECRET_ACCESS_KEY" = "",
    "AWS_S3_ENDPOINT"= "s3-h.ipe.df.gov.br"
)

OBJECT_OLD_KEY = 'DIEPS/bases/ped/PED-DF 2021.SAV'
OBJECT_NEW_KEY = 'DIEPS/bases/nova-ped-parquet/2021/PED-DF_2021.parquet'
BUCKET_NAME = 'raw'
OUTPUT_NAME = '/tmp/output.parquet'

In [ ]:
# Verifica conexão com o MinIO
bucketlist(region = '', use_https = TRUE)

In [ ]:
# Busca o objeto
tempfile = tempfile(fileext = '.sav')
object_path = paste0('s3://', BUCKET_NAME, '/', OBJECT_OLD_KEY)
filename = save_object(object = object_path, file = tempfile, region = '', use_https = TRUE)
filename

In [ ]:
data <- read_sav(filename)
head(data)

# Substituição de labels

As labels geradas pelo SPSS não tem um formato legal de serialização, então antes de armazenar os dados em parquet, seria interesante adicionar os dados em um formato de melhor serialização, como um dicionário.

In [ ]:
# Converte os campos categóricos em factor
data = as_factor(data, only_labelled = TRUE)
head(data)

In [ ]:
# Converte o dado para uma tabela do arrow
df2 <- arrow_table(data) 
df2

# Conversão para parquet

Converte o dataframe para parquet

In [ ]:
write_parquet(
    x = df2,
    sink = OUTPUT_NAME,
    compression = 'snappy',
    use_dictionary = TRUE
)

# Carregamento

Salva o arquivo de volta no bucket `raw`.

In [ ]:
put_object(
    bucket = BUCKET_NAME,
    object = OBJECT_NEW_KEY,
    file = OUTPUT_NAME,
    region = '',
    use_https = TRUE
)